In [4]:
import pandas as pd
import numpy as np
import cobra
import re

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import CustomSelectionIntegrationStrategy
from troppo.methods.reconstruction.fastcore import FASTcore, FastcoreProperties
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.gapfill.fastcc import FastCC, FastCCProperties
from cobra.io import read_sbml_model
import numpy as np

# parsing rule 
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

In [5]:
# load model and expression data
model = cobra.io.read_sbml_model('/home/biodata/aman/Human-GEM/model/Human-GEM.xml')
expression_data = pd.read_csv('/home/biodata/aman/data_processed/expression_data_ensembl.csv', index_col=0)

In [9]:
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000147576 or _ENSG00000172955 or _ENSG00000180011 or _ENSG00000187758 or _ENSG00000196344 or _ENSG00000196616 or _ENSG00000197894 or _ENSG00000198099 or _ENSG00000248144
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000117448
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000131069
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000111058 or _ENSG00000154930
  warnings.war

In [10]:
properties = FastCCProperties(
    flux_threshold = 1e-4,     # recommended: 1e-6 to 1e-4
    method = "original",       # "nonconvex" is slower, rare use
    solver = "CPLEX"           # or "gurobi", "glpk" if nothing else
)

# 3) run FastCC
fastcc = FastCC(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)
result_reactions = fastcc.run()     # list of flux-consistent reaction IDs

print(len(result_reactions))

[ 4149  4154  8019  9117  9685  9686 11744 12683]
12971 Total reactions
[12971 12970 12969 ...     7     6     5] Reversible reactions
6996 Irreversible reactions
LP7
-0.6307000000004016
11320 Flux consistent reactions, without flipping
689 Flux inconsistent irreversible reactions, without flipping
962 Flux inconsistent reactions, without flipping
LP7
-0.030999999999999868
11664 Flux consistent reactions
618 Flux inconsistent reversible reactions left to flip
LP7
-0.0001
11667 Flux consistent reactions
615 Flux inconsistent reversible reactions left to flip
LP7
0.0
615 reversible reactions flipped
LP7
-0.00030000000000000003
sanity check
11672 Flux consistent reactions
610 Flux inconsistent reversible reactions left to flip
LP7
-2.5102072113633578e-14
sanity check
11676 Flux consistent reactions
606 Flux inconsistent reversible reactions left to flip
LP7
9.094947017729282e-13
606 reversible reactions flipped
LP7
-9.094947017729282e-13
sanity check
11678 Flux consistent reactions
604 Fl

In [11]:
##############################################################
# 1. RUN FastCC, EXTRACT CONSISTENT REACTION IDs, CLEAN MODEL #
##############################################################

import numpy as np
from cobra.io import read_sbml_model

# ---- Extract original IDs before clean ----
all_rxns = [r.id for r in model.reactions]

# ---- Run FastCC ----
consistent_indices_raw = fastcc.run()

##########################################
# 2. FLATTEN CONSISTENT REACTION INDICES #
##########################################

consistent_indices = []

for x in consistent_indices_raw:
    # direct integer
    if isinstance(x, (int, np.integer)):
        consistent_indices.append(int(x))
    # nested list/array/tuple
    elif isinstance(x, (list, tuple, np.ndarray)):
        for y in x:
            if isinstance(y, (int, np.integer)):
                consistent_indices.append(int(y))

# remove duplicates, sort
consistent_indices = sorted(set(consistent_indices))

# convert to reaction IDs
consistent_rxns = [model.reactions[i].id for i in consistent_indices]

##############################################################
# 3. CLEAN MODEL: REMOVE REACTIONS NOT IN CONSISTENT SET     #
##############################################################

model = model.copy()

to_remove = [r.id for r in model.reactions if r.id not in consistent_rxns]

for rid in to_remove:
    try:
        model.reactions.get_by_id(rid).remove_from_model()
    except KeyError:
        pass  # already removed, ignore

############################################
# 4. SAVE CLEANED MODEL (USE JSON FORMAT!) #
############################################

#model.to_json("HumanGEM_fastcc_clean.json")

#############################
# 5. SANITY: BIOMASS CHECK  #
#############################

solution = model.optimize()
print("\n=== CLEANING FINISHED ===")
print("Total original reactions:", len(all_rxns))
print("Remaining reactions:", len(model.reactions))
print("Removed reactions:", len(to_remove))
print("Biomass flux after cleanup:", solution.objective_value)
print("==========================\n")


[ 4149  4154  8019  9117  9685  9686 11744 12683]
12971 Total reactions
[12971 12970 12969 ...     7     6     5] Reversible reactions
6996 Irreversible reactions
LP7
-0.6306999999995455
11326 Flux consistent reactions, without flipping
689 Flux inconsistent irreversible reactions, without flipping
956 Flux inconsistent reactions, without flipping
LP7
-0.0314999999997725
11669 Flux consistent reactions
613 Flux inconsistent reversible reactions left to flip
LP7
-0.00010000000002510208
11678 Flux consistent reactions
604 Flux inconsistent reversible reactions left to flip
LP7
0.0
11680 Flux consistent reactions
602 Flux inconsistent reversible reactions left to flip
LP7
4.547473508864641e-13
602 reversible reactions flipped
LP7
2.2737367544323206e-13
1 reversible reactions flipped
30 flux is inconsistent
1 reversible reactions flipped
197 flux is inconsistent
1 reversible reactions flipped
198 flux is inconsistent
1 reversible reactions flipped
294 flux is inconsistent
1 reversible reac

/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")



=== CLEANING FINISHED ===
Total original reactions: 12971
Remaining reactions: 11694
Removed reactions: 1277
Biomass flux after cleanup: 124.86814837744572



In [ ]:
expression_data

,expression
gene,
ENSG00000223972,0.000000
ENSG00000227232,0.015341
ENSG00000278267,0.000000
ENSG00000243485,0.000000
ENSG00000284332,0.000000
...,...
ENSG00000198727,9560.073000
ENSG00000210195,14.681607
ENSG00000210196,2.561994


In [12]:
expression_data.T

gene,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,NaN,ENSG00000240361,ENSG00000186092,ENSG00000233750,...,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196,ENSG00000275215,ENSG00000278233
expression,0.0,0.015341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046024,...,0.0,0.061365,856.93335,129.11223,1.488104,9560.073,14.681607,2.561994,0.0,0.0


In [13]:
#print(expression_data.head())
#expression_data_transposed = expression_data.T
#print(expression_data_transposed.head())

omics_container = TabularReader(path_or_df=expression_data.T, 
                                nomenclature='ensembl_gene_id',
                                omics_type='transcriptomics').to_containers()

single_sample = omics_container[0]

In [15]:
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

In [16]:
data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                 and_func=min, or_func=sum)

In [17]:
data_map.get_scores()

{'MAR03905': 78.286562788,
 'MAR03907': 44.24426,
 'MAR04097': 31.771797,
 'MAR04099': 98.04613599999999,
 'MAR04108': 31.771797,
 'MAR04133': 31.771797,
 'MAR04281': 506.59981,
 'MAR04388': 507.765747424,
 'MAR04283': 5.73763936,
 'MAR08357': 225.6702612,
 'MAR04379': 277.10958,
 'MAR04301': 277.10958,
 'MAR04355': 188.00741567,
 'MAR04358': 353.61657154,
 'MAR04360': None,
 'MAR04363': 674.694227,
 'MAR04365': 196.491128,
 'MAR04368': 154.47135642,
 'MAR04370': 134.68111499999998,
 'MAR04371': 196.491128,
 'MAR04372': 134.68111499999998,
 'MAR04373': 1215.1982752000001,
 'MAR04375': 802.45645567,
 'MAR04377': 0.85911185,
 'MAR04381': 136.00047,
 'MAR04391': 305.41425,
 'MAR04394': 113.80163007,
 'MAR04396': 30.728589,
 'MAR04521': 29.82345269,
 'MAR06412': 20.081738,
 'MAR07747': 39.304367,
 'MAR08360': 46.637497,
 'MAR08757': 11.996882,
 'MAR03989': 108.3554822,
 'MAR04122': 10.508779,
 'MAR04837': 8.913284529999999,
 'MAR05395': 29.6853806,
 'MAR05396': 53.24959067,
 'MAR09727': 43

In [18]:
# model = model_wrapper.model_reader.model

# # search for biomass in reaction names
# for r in model.reactions:
#     if 'biomass' in r.name.lower() or 'growth' in r.name.lower():
#         print(r.id, r.name)

In [19]:
from math import log
threshold =  (0.2 * log(2))
protected_reactions = ["MAR04413", "MAR09931", "MAR09932","MAR10024","MAR10062", "MAR10063", "MAR10064", "MAR10065","MAR13082"]

def integration_fx(reaction_map_scores):
    return [[k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > threshold) or k in protected_reactions]]

threshold_integration = CustomSelectionIntegrationStrategy(group_functions=[integration_fx])
threshold_scores = threshold_integration.integrate(data_map=data_map)

print(threshold_scores)
print([len(i) for i in threshold_scores])

[['MAR03905', 'MAR03907', 'MAR04097', 'MAR04099', 'MAR04108', 'MAR04133', 'MAR04281', 'MAR04388', 'MAR04283', 'MAR08357', 'MAR04379', 'MAR04301', 'MAR04355', 'MAR04358', 'MAR04363', 'MAR04365', 'MAR04368', 'MAR04370', 'MAR04371', 'MAR04372', 'MAR04373', 'MAR04375', 'MAR04377', 'MAR04381', 'MAR04391', 'MAR04394', 'MAR04396', 'MAR04521', 'MAR06412', 'MAR07747', 'MAR08360', 'MAR08757', 'MAR03989', 'MAR04122', 'MAR04837', 'MAR05395', 'MAR05396', 'MAR09727', 'MAR05397', 'MAR05398', 'MAR05399', 'MAR05400', 'MAR05401', 'MAR08568', 'MAR08569', 'MAR08570', 'MAR08571', 'MAR08572', 'MAR08573', 'MAR08574', 'MAR08575', 'MAR08576', 'MAR08577', 'MAR08578', 'MAR08579', 'MAR08581', 'MAR08591', 'MAR08592', 'MAR08580', 'MAR08582', 'MAR08587', 'MAR08589', 'MAR08583', 'MAR08584', 'MAR08585', 'MAR03944', 'MAR04128', 'MAR04130', 'MAR04131', 'MAR04132', 'MAR04303', 'MAR04414', 'MAR04416', 'MAR07674', 'MAR08761', 'MAR08762', 'MAR08766', 'MAR08767', 'MAR04297', 'MAR04315', 'MAR04316', 'MAR04319', 'MAR04383', 'M

In [20]:
# Get the index of the reaction of the CORE reaction set
ordered_ids = {r:i for i,r in enumerate(model_wrapper.model_reader.r_ids)}
core_idx = [[ordered_ids[k] for k in l] for l in threshold_scores]
print([len(i) for i in core_idx])

[6873]


In [21]:
# # 1) load your model
# model = read_sbml_model("Human-GEM.xml")

# # 2) extract S, bounds, properties
# S = model.sigma  # stoichiometric matrix (cobra gives it)
# lb = np.array([r.lower_bound for r in model.reactions])
# ub = np.array([r.upper_bound for r in model.reactions])

properties = FastCCProperties(
    flux_threshold = 1e-4,     # recommended: 1e-6 to 1e-4
    method = "original",       # "nonconvex" is slower, rare use
    solver = "CPLEX"           # or "gurobi", "glpk" if nothing else
)

# 3) run FastCC
fastcc = FastCC(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)
result_reactions = fastcc.run()     # list of flux-consistent reaction IDs

print(len(result_reactions))

[]
11694 Total reactions
[11694 11693 11692 ...     6     5     4] Reversible reactions
6307 Irreversible reactions
LP7
-0.6306999999999469
11275 Flux consistent reactions, without flipping
419 Flux inconsistent reactions, without flipping
LP7
-0.03539999999999999
11667 Flux consistent reactions
27 Flux inconsistent reversible reactions left to flip
LP7
0.0
11671 Flux consistent reactions
23 Flux inconsistent reversible reactions left to flip
LP7
0.0
11673 Flux consistent reactions
21 Flux inconsistent reversible reactions left to flip
LP7
0.0
21 reversible reactions flipped
LP7
-0.00030000000000000003
sanity check
11680 Flux consistent reactions
14 Flux inconsistent reversible reactions left to flip
LP7
0.0
sanity check
11682 Flux consistent reactions
12 Flux inconsistent reversible reactions left to flip
LP7
0.0
sanity check
11684 Flux consistent reactions
10 Flux inconsistent reversible reactions left to flip
LP7
0.0
10 reversible reactions flipped
LP7
0.0
sanity check
11686 Flux co

In [25]:
# Define the FastCORE properties
properties = FastcoreProperties(core=core_idx, solver='CPLEX')

# instantiate the FastCORE class
fastcore = FASTcore(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

# Run the algorithm
model_fastcore = fastcore.run()

J size4866
[    0     1     2 ... 11688 11689 11690]
before LP7
LP7
-0.48659999999996273
done LP7
LP9
54625081.347482964
Warning, Solution is not optimal
done LP9
449 8483
before LP7
LP7
-0.04080000000000014
done LP7
LP9
6175.000000011409
Warning, Solution is not optimal
done LP9
449 9519


ValueError: operands could not be broadcast together with shapes (426,) (0,) 

In [22]:
pd.DataFrame(model_fastcore)

NameError: name 'model_fastcore' is not defined

In [ ]:
# variable will contain the indices of the reactions that should be kept in the final model. 

In [ ]:
print(type(model_fastcore))

<class 'list'>


In [ ]:
pd.DataFrame(model_fastcore).to_csv('fastcore_output_tinit.tsv', sep='\t')

---

In [ ]:
# model.reactions
# len(model.reactions)

In [ ]:
selected_reactions = [model.reactions[i] for i in model_fastcore]
selected_reactions

[]

---

In [ ]:
ctx_model = model.copy()

selected_ids = [r.id for r in selected_reactions]
to_remove = [r for r in ctx_model.reactions if r.id not in selected_ids]

ctx_model.remove_reactions(to_remove, remove_orphans=True)

ctx_model

/home/biodata/miniconda3/envs/cplex_aman/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Name,HumanGEM
Memory address,7ac63efd7bb0
Number of metabolites,0
Number of reactions,0
Number of genes,0
Number of groups,148
Objective expression,0
Compartments,


In [ ]:
len(model.reactions), len(ctx_model.reactions)

(12971, 0)

In [ ]:
# export
cobra.io.write_sbml_model(ctx_model, "fastcore_context_specific_model.xml")

In [ ]:
solution = ctx_model.optimize()
print(solution.objective_value)

0.0


In [ ]:
ctx_model.summary()

Non-linear or non-reaction model objective. Falling back to minimal display.


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux
